In [ ]:
import os
import re
import sys
import glob
import pickle
import tables
from tqdm import tqdm
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt, hilbert

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook

import tensorflow as tf
from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import read_area_values, load_data_areas, load_data_slide
from dlml.nn import predict, compute_receptive_field, compute_correlations

#### A function for plotting the correlations analaysis results

In [ ]:
def plot_correlations(R, p, edges, F, Xf, idx, sort_F=1.0, vmin=None, vmax=None):
    if p is not None:
        R_sig = R.copy()
        R_sig[p > 0.05] = 0
        R_mean = R_sig.mean(axis=0)
    else:
        R_mean = R.mean(axis=0)

    edge = np.abs(edges - sort_F).argmin()
    kdx = np.argsort(R_mean[edge,:])
    R_mean = R_mean[:,kdx]

    fig = plt.figure(figsize=(10,5))
    gs = fig.add_gridspec(1, 4)
    ax = [fig.add_subplot(gs[0,0]), fig.add_subplot(gs[0,1:])]
    cmap = plt.get_cmap('tab10', 2)
    for j,jdx in enumerate(idx):
        mean = Xf[jdx].mean(axis=0)
        stddev = Xf[jdx].std(axis=0)
        ci = 1.96 * stddev / np.sqrt(jdx.size)
        ax[0].fill_betweenx(F, mean + ci, mean - ci, color=cmap(j))
    ax[0].set_yscale('log')
    ax[0].invert_xaxis()
    ax[0].yaxis.tick_right()
    cmap = plt.get_cmap('bwr')
    make_symmetric = False
    if vmin is None:
        vmin = R_mean.min()
        make_symmetric = True
    if vmax is None:
        vmax = R_mean.max()
        if make_symmetric:
            if vmax > np.abs(vmin):
                vmin = -vmax
            else:
                vmax = -vmin
    print(f'Color bar bounds: ({vmin:.2f},{vmax:.2f}).')
    x = np.arange(R_mean.shape[-1])
    y = edges[:-1] + np.diff(edges) / 2
    im = ax[1].pcolormesh(x, y, R_mean, vmin=vmin, vmax=vmax, shading='auto', cmap=cmap)
    ax[1].set_yscale('log')
    cbar = plt.colorbar(im, label='Correlation')
    ax[1].set_xlabel('Filter #')
    ax[1].set_ylabel('Frequency [Hz]')
    for a in ax:
        a.set_ylim(edges[[0,-2]])
    for side in 'left','top':
        ax[0].spines[side].set_visible(False)
    ax[0].set_yticklabels([])
    for side in 'right','top':
        ax[1].spines[side].set_visible(False)
    fig.tight_layout()
    return fig, vmin, vmax

#### Find the best experiment given a set of tags

In [ ]:
area_ID = 1
area_measure = 'momentum'
stoch_load_bus_IDs = []
rec_bus_IDs = [3]
H_G1, D, DZA = None, None, None # 500, 2, 0
additional_tags = ['ReLU_none', 'converted_from_PowerFactory', 'all_stoch_loads', 'data_subset']
missing_tags = []
use_FFT = False
if use_FFT:
    additional_tags.append('fft')
else:
    missing_tags.append('fft')
pooling_type = ''
if pooling_type is not None and pooling_type != '':
    additional_tags.append(pooling_type + '_pooling')

# training on frequency data, 2 output values
# experiment_ID = '9ea493c789b542bf979c51a6031f4044'
# training on frequency data, 4 output values
# experiment_ID = 'f6d9a03f1cfe450288e9cb86da94235f'
# training on time series data, 2 output values
experiment_ID = '034a1edb0797475b985f0e1335dab383'
# training on time series data, 4 output values
# experiment_ID = 'b346a89d384c4db2ba4058a2c83c4f12'
# training on time series data, 2 output values, with MaxPooling1DWithArgmax layer
# experiment_ID = '9034f8bc4f874c938dfa5f1f9ee04e82'
# experiment_ID = None

if experiment_ID is not None:
    from comet_ml.api import API, APIExperiment
    api = API(api_key = os.environ['COMET_API_KEY'])
    experiment = api.get_experiment('danielelinaro', 'inertia', experiment_ID)
    sys.stdout.write(f'Getting metrics for experiment {experiment_ID[:6]}... ')
    sys.stdout.flush()
    metrics = experiment.get_metrics()
    sys.stdout.write('done.\n')
    val_loss = []
    for m in metrics:
        if m['metricName'] == 'val_loss':
            val_loss.append(float(m['metricValue']))
        elif m['metricName'] == 'mape_prediction':
            MAPE = float(m['metricValue'])
    val_loss = np.array(val_loss)
else:
    # find the best experiment that matches the set of tags above
    experiments = collect_experiments(area_ID, area_measure=area_measure, D=D, DZA=DZA, \
                                      stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=H_G1, \
                                      rec_bus_IDs=rec_bus_IDs, additional_tags=additional_tags, \
                                      missing_tags=missing_tags, verbose=True)
    experiment_IDs = list(experiments.keys())
    experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
#     experiment_ID = experiment_IDs[np.argmin([expt['MAPE'] for expt in experiments.values()])]

    MAPE = experiments[experiment_ID]['MAPE']
    loss = experiments[experiment_ID]['loss']
    val_loss = experiments[experiment_ID]['val_loss']
    batch_loss = experiments[experiment_ID]['batch_loss']
    tags = experiments[experiment_ID]['tags']
print(f'Selected experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE:.4f}%).')

#### Load the model

In [ ]:
experiments_path = '../experiments/neural_network/'
network_parameters = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'parameters.pkl'), 'rb'))
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
try:
    epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
    best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
except:
    best_checkpoint = checkpoint_files[-1]
try:
    model = keras.models.load_model(best_checkpoint)
except:
    if pooling_type == 'downsample':
        custom_objects = {'DownSampling1D': DownSampling1D}
    elif pooling_type == 'spectral':
        custom_objects = {'SpectralPooling': SpectralPooling}
    elif pooling_type == 'argmax':
        custom_objects = {'MaxPooling1DWithArgmax': MaxPooling1DWithArgmax}
    with keras.utils.custom_object_scope(custom_objects):
        model = keras.models.load_model(best_checkpoint)
if pooling_type == 'argmax':
    for layer in model.layers:
        if isinstance(layer, MaxPooling1DWithArgmax):
            print(f'Setting store_argmax = True for layer "{layer.name}".')
            layer.store_argmax = True
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
try:
    x_train_min = network_parameters['x_train_min']
    x_train_max = network_parameters['x_train_max']
except:
    pass
var_names = network_parameters['var_names']
print(f'Loaded network from {best_checkpoint}.')
print(f'Variable names: {var_names}')

#### Plot the model topology

In [ ]:
model.summary()

### Effective receptive field size and stride of the model

In [ ]:
effective_RF_size,effective_stride = compute_receptive_field(model, stop_layer=keras.layers.Flatten)
print('Effective receptive field size:')
for i,(k,v) in enumerate(effective_RF_size.items()):
    print(f'{i}. {k} ' + '.' * (20 - len(k)) + ' {:d}'.format(v))
print()
print('Effective stride:')
for i,(k,v) in enumerate(effective_stride.items()):
    print(f'{i}. {k} ' + '.' * (20 - len(k)) + ' {:d}'.format(v))

#### Load the data set

In [ ]:
set_name = 'test'
use_fft = network_parameters['use_fft'] if 'use_fft' in network_parameters else False
data_dirs = []
for area_ID,data_dir in zip(network_parameters['area_IDs'], network_parameters['data_dirs']):
    data_dirs.append(data_dir.format(area_ID))
data_dir = os.path.join('..', data_dirs[0])
data_files = sorted(glob.glob(data_dir + os.path.sep + f'*_{set_name}_set.h5'))
ret = load_data_areas({set_name: data_files}, network_parameters['var_names'],
                        network_parameters['generators_areas_map'][:1],
                        network_parameters['generators_Pnom'],
                        network_parameters['area_measure'],
                        trial_dur=network_parameters['trial_duration'],
                        max_block_size=100,
                        use_tf=False, add_omega_ref=True,
                        use_fft=use_fft)
y = ret[2][set_name]
if use_fft:
    X = [(ret[1][set_name][i] - m) / (M - m) for i,(m,M) in enumerate(zip(x_train_min, x_train_max))]
    F = ret[0]
else:
    X = [(ret[1][set_name][i] - m) / s for i,(m,s) in enumerate(zip(x_train_mean, x_train_std))]
    t = ret[0]

#### Predict the momentum using the model

In [ ]:
IDX = [np.where(y == mom)[0] for mom in np.unique(y)]
n_mom_values = len(IDX)
momentum = [np.squeeze(model.predict(X[0][jdx])) for jdx in IDX]
mean_momentum = [m.mean() for m in momentum]
stddev_momentum = [m.std() for m in momentum]
print('Mean momentum:', mean_momentum)
print(' Std momentum:', stddev_momentum)

#### Clone the above model
This initializes the cloned model with new random weights and will be used in the following as a control for the correlation analysis

In [ ]:
reinit_model = keras.models.clone_model(model)
reinit_momentum = [np.squeeze(reinit_model.predict(X[0][jdx])) for jdx in IDX]
mean_reinit_momentum = [m.mean() for m in reinit_momentum]
stddev_reinit_momentum = [m.std() for m in reinit_momentum]
print('Mean momentum:', mean_reinit_momentum)
print(' Std momentum:', stddev_reinit_momentum)

### Plot the inputs and their FFTs

In [ ]:
cmap = plt.get_cmap('tab10', n_mom_values)
if use_fft:
    Xf = X
else:
    data_files_training = sorted(glob.glob(data_dir + os.path.sep + f'*_training_set.h5'))
    if len(data_files_training) == 0:
        data_files_training = sorted(glob.glob(data_dir + os.path.sep + f'*_test_set.h5'))
    ret_fft = load_data_areas({'training': data_files_training}, network_parameters['var_names'],
                        network_parameters['generators_areas_map'][:1],
                        network_parameters['generators_Pnom'],
                        network_parameters['area_measure'],
                        trial_dur=network_parameters['trial_duration'],
                        max_block_size=200,
                        use_tf=False, add_omega_ref=True,
                        use_fft=True)
    x_train_min_fft = np.array([val.min() for val in ret_fft[1]['training']], dtype=np.float32)
    x_train_max_fft = np.array([val.max() for val in ret_fft[1]['training']], dtype=np.float32)
    ret = load_data_areas({set_name: data_files}, network_parameters['var_names'],
                        network_parameters['generators_areas_map'][:1],
                        network_parameters['generators_Pnom'],
                        network_parameters['area_measure'],
                        trial_dur=network_parameters['trial_duration'],
                        max_block_size=100,
                        use_tf=False, add_omega_ref=True,
                        use_fft=True)
    F = ret[0]
    Xf = [(ret[1][set_name][i] - m) / (M - m) for i,(m,M) in enumerate(zip(x_train_min_fft,
                                                                           x_train_max_fft))]

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(10, 4))

for j,jdx in enumerate(IDX):
    mean = X[0][jdx].mean(axis=0)
    stddev = X[0][jdx].std(axis=0)
    ci = 1.96 * stddev / np.sqrt(jdx.size)
    ax[0].fill_between(t, mean + ci, mean - ci, color=cmap(j))
    mean = Xf[0][jdx].mean(axis=0)
    stddev = Xf[0][jdx].std(axis=0)
    ci = 1.96 * stddev / np.sqrt(jdx.size)
    ax[1].fill_between(F, mean + ci, mean - ci, color=cmap(j))

for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.grid(which='major', axis='both', ls=':', lw=0.5, color=[.6,.6,.6])
ax[1].set_xscale('log')
ax[0].set_xlabel('Time [min]')
ax[0].set_ylabel('Normalized trace')
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel('Normalized FFT')
fig.tight_layout()
fig.savefig(f'spectra_{n_mom_values}_momentum_levels.pdf')

### Build a model with as many outputs as there are convolutional or pooling layers

In [ ]:
outputs = [layer.output for layer in model.layers \
           if layer.name in effective_RF_size.keys() and not isinstance(layer, keras.layers.InputLayer)]
multi_output_model = keras.Model(inputs=model.inputs, outputs=outputs)
print(f'The model has {len(outputs)} outputs, corresponding to the following layers:')
for i,layer in enumerate(multi_output_model.layers):
    if not isinstance(layer, keras.layers.InputLayer):
        print(f'    {i}. {layer.name}')

### Correlations in the actual model

Define some variables used here and for the control model below

In [ ]:
sort_F = 1.1
dt = np.diff(t[:2])[0]
edges = np.logspace(np.log10(0.05), np.log10(0.5 / dt), 50)
bands = [[a,b] for a,b in zip(edges[:-1], edges[1:])]
N_bands = len(bands)
_, N_neurons, N_filters = multi_output_model.layers[-1].output.shape
N_trials = X[0].shape[0]
output_file = f'correlations_{experiment_ID[:6]}_{N_bands}-bands_' + \
    f'{N_filters}-filters_{N_neurons}-neurons_{N_trials}-trials'

Compute the correlations:

In [ ]:
force = False
if not os.path.isfile(output_file + '.npz') or force:
    R,p = compute_correlations(multi_output_model, X[0], dt, bands, effective_RF_size, effective_stride)
    np.savez_compressed(output_file + '.npz', R=R, p=p, edges=edges)
else:
    data = np.load(output_file + '.npz')
    R = data['R']
    p = data['p']
    edges = data['edges']

Plot the results:

In [ ]:
fig,vmin,vmax = plot_correlations(R, p, edges, F, Xf[0], IDX, sort_F=sort_F)
fig.savefig(output_file + '.pdf')

### Correlations in a control model with random weights

Build a control model with the same (multiple-output) architecture as the previous one but random weights:

In [ ]:
outputs = [layer.output for layer in reinit_model.layers \
           if layer.name in effective_RF_size.keys() and not isinstance(layer, keras.layers.InputLayer)]
ctrl_model = keras.Model(inputs=reinit_model.inputs, outputs=outputs)

Compute the correlations:

In [ ]:
force = False
if not os.path.isfile(output_file + '_CTRL.npz') or force:
    R_ctrl,p_ctrl = compute_correlations(ctrl_model, X[0], dt, bands, effective_RF_size, effective_stride)
    np.savez_compressed(output_file + '_CTRL.npz', R=R_ctrl, p=p_ctrl, edges=edges)
else:
    data = np.load(output_file + '_CTRL.npz')
    R_ctrl = data['R']
    p_ctrl = data['p']
    edges = data['edges']

Plot the results:

In [ ]:
fig,vmin,vmax = plot_correlations(R_ctrl, p_ctrl, edges, F, Xf[0], IDX, sort_F=sort_F, vmin=vmin, vmax=vmax)
fig.savefig(output_file + '_CTRL.pdf')